In [46]:
import functools
import operator
import itertools
import math
import sys
import json
import re
from enum import Enum
from collections import deque
from dataclasses import dataclass
from typing import Dict, List, Tuple, TypeAlias, Set
import pulp
Coords: TypeAlias = Tuple[int, int]

data = open("input.txt").read().splitlines()
grid = [[x for x in line] for line in data]

directions = {
    (-1, 0): [(-1, -1), (-1, 0), (-1, 1)],
    (0, 1): [(-1, 1), (0, 1), (1, 1)],
    (1, 0): [(1, -1), (1, 0), (1, 1)],
    (0, -1): [(1, -1), (0, -1), (-1, -1)],
}


def add(a, b):
    return (a[0]+b[0], a[1]+b[1])


class Elf():
    def __init__(self, coords: Tuple[int, int]) -> None:
        self.coords = coords

    def __repr__(self) -> str:
        return f"{self.coords}"

    def propose(self, dir_seq: List[Coords], others: Dict[Coords, "Elf"]) -> Coords:
        
        isolated = True
        for dirs in directions.values():
            for d in dirs:
                if others.get(add(self.coords, d)):
                    isolated = False
                    break
        
        if isolated:
            return None

        for d in dir_seq:
            dirs = directions[d]
            if all([not others.get(add(self.coords, dir)) for dir in dirs]):
                return add(self.coords, d)
        return None

    def move(self, coords: Coords, mapping: Dict[Coords, "Elf"]):
        mapping.pop(self.coords)
        self.coords = coords
        mapping[self.coords] = self


elves: List[Elf] = []
elf_map: Dict[Coords, Elf] = {}
for r, row in enumerate(grid):
    for c, char in enumerate(row):
        if char == "#":
            elf = Elf((r, c))
            elves.append(elf)
            elf_map[(r, c)] = elf


def get_corners(ls: List[Coords]) -> Tuple[int, int, int, int]:
    """Returns (min_r, max_r, min_c, max_c)"""
    min_r = math.inf
    max_r = 0
    min_c = math.inf
    max_c = 0
    for r, c in ls:
        min_r = min(min_r, r)
        max_r = max(max_r, r)
        min_c = min(min_c, c)
        max_c = max(max_c, c)
    return (min_r, max_r, min_c, max_c)


def get_dimensions(ls: List[Coords]) -> Tuple[int, int]:
    """returns (height, width)"""
    min_r, max_r, min_c, max_c = get_corners(ls)
    return (max_r - min_r + 1, max_c - min_c + 1)


def get_area(ls: List[Coords]) -> int:
    h, w = get_dimensions(ls)
    return h * w


def get_unoccupied(ls: List[Coords]) -> int:
    area = get_area(ls)
    return area - len(ls)


def draw(m: Dict[Coords, Elf]):
    min_r, max_r, min_c, max_c = get_corners(m.keys())
    for r in range(min_r, max_r + 1):
        for c in range(min_c, max_c + 1):
            if (r, c) in m:
                print("#", end="")
            else:
                print(".", end="")
        print()


#draw(elf_map)
#print(get_unoccupied(elf_map.keys()))
dir_seq = deque([
    (-1, 0),
    (1, 0),
    (0, -1),
    (0, 1),
])

def part_1():
    for i in range(0, 10):
        proposal_map: Dict[Coords, List[Elf]] = {}
        for elf in elves:
            proposal = elf.propose(dir_seq, elf_map)
            ls = proposal_map.get(proposal, [])
            ls.append(elf)
            proposal_map[proposal] = ls
        
        for proposal,ls in proposal_map.items():
            if len(ls) != 1:
                continue
            elf = ls[0]
            elf.move(proposal, elf_map)
        prev = dir_seq.popleft()
        dir_seq.append(prev)

        #print(f"------- {i} --------")    
        #draw(elf_map)
        keys = elf_map.keys()
        a,b,c,d = get_corners(keys)
        height,width = get_dimensions(keys)
        area = get_area(keys)
        unoccupied = get_unoccupied(keys)
        #print(f"corners: {(a,b,c,d)}")
        #print(f"{height} x {width} = {area}")
        #print(f"unoccupied = {unoccupied}")

    print(get_unoccupied(elf_map.keys()))
#part_1()

def part_2():
    count = 0
    while True:
        count += 1
        proposal_map: Dict[Coords, List[Elf]] = {}
        for elf in elves:
            proposal = elf.propose(dir_seq, elf_map)
            if proposal:
                ls = proposal_map.get(proposal, [])
                ls.append(elf)
                proposal_map[proposal] = ls
        
        if len(proposal_map) == 0:
            break

        for proposal,ls in proposal_map.items():
            if len(ls) != 1:
                continue
            elf = ls[0]
            elf.move(proposal, elf_map)
        prev = dir_seq.popleft()
        dir_seq.append(prev)

        if count % 100 == 0:
            print(f"------- {count} --------")    
            draw(elf_map)
            keys = elf_map.keys()
            a,b,c,d = get_corners(keys)
            height,width = get_dimensions(keys)
            area = get_area(keys)
            unoccupied = get_unoccupied(keys)
            print(f"corners: {(a,b,c,d)}")
            print(f"{height} x {width} = {area}")
            print(f"unoccupied = {unoccupied}")

    print(get_unoccupied(elf_map.keys()))
    print(count)
part_2()


------- 100 --------
....................................#............................................................
............................#.#.#.................#......................#.......................
......................#.#..........#......#....#.........#.................#.....................
.................................#.....#....#......#.#.....#.#..#.#..........#...................
.......................#...#.#.......#..........#.....................#..#.#.....................
................#...#....#.....#.#..#...#.....#.#....#.#....#..##.#.#........#....#..............
.............#........#.....#.........#...#...............#...........#.........#................
...............#....#...#.#...#..#......#...#....#..#........#...#......#..#....#...#............
.....#.....#............#...#.........#.....#.#....#...#...#.......#..#.......#..................
...#....#.....#...#..#......#..###..#.#.#...#.#.#...#.#......#.#.....#....#.#...#......#.........